In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch
import pandas as pd
import time

c:\Users\Guille\anaconda3\envs\Sentiment\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
rest_google_reviews = pd.read_csv("../ML/Google_review_test.csv",delimiter=",")
rest_google_reviews.head()

,Unnamed: 0,user_id,name_x,text
0,0,104959831058524862747,Samuel Benson,A pretty seamless experience from top to botto...
1,1,100925653611153628883,Emily Probst,Great food and great service! Being so close t...
2,2,100496729305505499660,Brianna Dungan,Pleasant location.
3,3,115619895645642399701,Aaron Tim,Finally a wingstop in Provo
4,4,115522360097670804488,Heather Ann Bruin,NaN


In [4]:
print("CUDA disponible:", torch.cuda.is_available())

CUDA disponible: True


In [5]:
# Verifica si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [6]:
# Carga modelos a la GPU a excepcion del modelo de tokens
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/beto-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/beto-sentiment-analysis")
model = model.to(device)
pipe_beto = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer, device=0 if str(device) == "cuda" else -1)

In [11]:
# Función para procesar la reseña y mostrar progreso cada 1 minuto
def sentiment_status(review):
    if review is None:
        return None
    else:
        text = str(review)[:512]
        clasificacion = pipe_beto(text)
        return clasificacion[0]['label']

# Contador de reviews procesados
elementos_procesados = 0

# Tiempo inicial
tiempo_inicial = time.time()

# Itera sobre los reviews y muestra el progreso cada 1 minuto
for index, row in rest_google_reviews.iterrows():
    rest_google_reviews.at[index, 'sentiment'] = sentiment_status(row['text'])
    elementos_procesados += 1
    
    # Calcula el tiempo transcurrido
    tiempo_transcurrido = time.time() - tiempo_inicial
    
    # Calcula el tiempo estimado restante
    elementos_restantes = len(rest_google_reviews) - elementos_procesados
    tiempo_restante = (tiempo_transcurrido / elementos_procesados) * elementos_restantes
    
    # Verifica si ha pasado 1 minuto
    if tiempo_transcurrido >= 60:
        tiempo_inicial = time.time()
        print(f"Procesados: {elementos_procesados}/{len(rest_google_reviews)}, % completado: {round(elementos_procesados/len(rest_google_reviews)*100, 2)}%, Tiempo estimado restante: {round((tiempo_restante/60)/60,2)} horas")

# Mensaje final
print("Procesamiento completado.")

c:\Users\Guille\anaconda3\envs\Sentiment\lib\site-packages\transformers\pipelines\base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Procesados: 6830/152797, % completado: 4.47%, Tiempo estimado restante: 0.36 horas
Procesados: 13744/152797, % completado: 8.99%, Tiempo estimado restante: 0.17 horas
Procesados: 20640/152797, % completado: 13.51%, Tiempo estimado restante: 0.11 horas
Procesados: 27543/152797, % completado: 18.03%, Tiempo estimado restante: 0.08 horas
Procesados: 34417/152797, % completado: 22.52%, Tiempo estimado restante: 0.06 horas
Procesados: 41259/152797, % completado: 27.0%, Tiempo estimado restante: 0.05 horas
Procesados: 48015/152797, % completado: 31.42%, Tiempo estimado restante: 0.04 horas
Procesados: 54745/152797, % completado: 35.83%, Tiempo estimado restante: 0.03 horas
Procesados: 61623/152797, % completado: 40.33%, Tiempo estimado restante: 0.02 horas
Procesados: 68418/152797, % completado: 44.78%, Tiempo estimado restante: 0.02 horas
Procesados: 75252/152797, % completado: 49.25%, Tiempo estimado restante: 0.02 horas
Procesados: 82021/152797, % completado: 53.68%, Tiempo estimado resta

In [12]:
rest_google_reviews.head(20)

,Unnamed: 0,user_id,name_x,text,sentiment
0,0,104959831058524862747,Samuel Benson,A pretty seamless experience from top to botto...,NEU
1,1,100925653611153628883,Emily Probst,Great food and great service! Being so close t...,POS
2,2,100496729305505499660,Brianna Dungan,Pleasant location.,NEU
3,3,115619895645642399701,Aaron Tim,Finally a wingstop in Provo,NEU
4,4,115522360097670804488,Heather Ann Bruin,NaN,NEU
5,5,111139967532022596352,Hootie Hennessey,NaN,NEU
6,6,104959831058524862747,Samuel Benson,A pretty seamless experience from top to botto...,NEU
7,7,100925653611153628883,Emily Probst,Great food and great service! Being so close t...,POS
8,8,100496729305505499660,Brianna Dungan,Pleasant location.,NEU
9,9,115619895645642399701,Aaron Tim,Finally a wingstop in Provo,NEU
